ссылка на github:
https://github.com/ 

# Анализ лояльности пользователей Яндекс Афиши с помощью Python

### Установка библиотек. Настройка подключения к БД. Основной запрос данных и датафрейм с курсом тенге

In [50]:
!pip install sqlalchemy
!pip install psycopg2

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [36]:
import pandas as pd
from sqlalchemy import create_engine

In [37]:
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-afisha' # название базы данных
             }

In [38]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'],
)

In [39]:
engine = create_engine(connection_string)

In [44]:
query = '''
SELECT 
	p.user_id, p.device_type_canonical,	p.order_id, p.created_dt_msk::date order_dt, p.created_ts_msk order_ts, p.currency_code, p.revenue, p.tickets_count,
	EXTRACT (DAY FROM p.created_dt_msk - lag(p.created_dt_msk) over(PARTITION BY p.user_id ORDER BY p.created_dt_msk)) days_since_prev,
	p.event_id, e.event_name_code event_name, p.service_name, e.event_type_main, r.region_name, c.city_name
FROM afisha.purchases p JOIN afisha.events e using(event_id) JOIN afisha.city c using(city_id) JOIN afisha.regions r using(region_id)
WHERE device_type_canonical IN ('mobile','desktop') 
	AND event_type_main <> 'фильм'
ORDER BY user_id;
'''

In [47]:
df = pd.read_sql_query(query, con=engine)

In [55]:
df.info()
print(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290611 entries, 0 to 290610
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   user_id                290611 non-null  object        
 1   device_type_canonical  290611 non-null  object        
 2   order_id               290611 non-null  int64         
 3   order_dt               290611 non-null  object        
 4   order_ts               290611 non-null  datetime64[ns]
 5   currency_code          290611 non-null  object        
 6   revenue                290611 non-null  float64       
 7   tickets_count          290611 non-null  int64         
 8   days_since_prev        268678 non-null  float64       
 9   event_id               290611 non-null  int64         
 10  event_name             290611 non-null  object        
 11  service_name           290611 non-null  object        
 12  event_type_main        290611 non-null  obje

In [52]:
tenge = pd.read_csv('https://code.s3.yandex.net/datasets/final_tickets_tenge_df.csv')

In [54]:
tenge.info()
print(tenge)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 357 entries, 0 to 356
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   data     357 non-null    object 
 1   nominal  357 non-null    int64  
 2   curs     357 non-null    float64
 3   cdx      357 non-null    object 
dtypes: float64(1), int64(1), object(2)
memory usage: 11.3+ KB
           data  nominal     curs  cdx
0    2024-01-10      100  19.9391  kzt
1    2024-01-11      100  19.7255  kzt
2    2024-01-12      100  19.5839  kzt
3    2024-01-13      100  19.4501  kzt
4    2024-01-14      100  19.4501  kzt
..          ...      ...      ...  ...
352  2024-12-27      100  19.2705  kzt
353  2024-12-28      100  19.5105  kzt
354  2024-12-29      100  19.4860  kzt
355  2024-12-30      100  19.4860  kzt
356  2024-12-31      100  19.4860  kzt

[357 rows x 4 columns]
